# Liver Dataset w/ Decision Trees

In [15]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

# fetch dataset 
ilpd_indian_liver_patient_dataset = fetch_ucirepo(id=225) 
  
# data (as pandas dataframes) 
X = ilpd_indian_liver_patient_dataset.data.features 
y = ilpd_indian_liver_patient_dataset.data.targets 

In [16]:
print(X.head())
print(X.isnull().sum())

   Age  Gender    TB   DB  Alkphos  Sgpt  Sgot   TP  ALB  A/G Ratio
0   65  Female   0.7  0.1      187    16    18  6.8  3.3       0.90
1   62    Male  10.9  5.5      699    64   100  7.5  3.2       0.74
2   62    Male   7.3  4.1      490    60    68  7.0  3.3       0.89
3   58    Male   1.0  0.4      182    14    20  6.8  3.4       1.00
4   72    Male   3.9  2.0      195    27    59  7.3  2.4       0.40
Age          0
Gender       0
TB           0
DB           0
Alkphos      0
Sgpt         0
Sgot         0
TP           0
ALB          0
A/G Ratio    4
dtype: int64


##### Description of Variables
1. Age - Age of the patient
2. Gender - Gender of the patient
3. TB - Total Bilirubin
4. DB - Direct Bilirubin
5. Alkphos - Alkaline Phosphotase
6. Sgpt - Alamine Aminotransferase
7. Sgot - Aspartate Aminotransferase
8. TP - Total Proteins
9. ALB - Albumin
10. A/G Ratio - Albumin and Globulin Ratio

In [17]:
# One-hot encode the 'Gender' variable
encoder = LabelEncoder()
X.loc[:, 'Gender'] = encoder.fit_transform(X['Gender'])

# Fill in missing values for the 'A/G Ratio' column
X.loc[:, 'A/G Ratio'] = X['A/G Ratio'].fillna(X['A/G Ratio'].mean())

# Reset indices of X and y before dropping duplicates
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

# Drop duplicates in X and align y accordingly
X = X.drop_duplicates()
y = y.loc[X.index]  # Align y with the updated indices of X

# Ensure y is flattened to 1-dimensional
y = y.squeeze()  # Convert to a Series if it's multidimensional

# Fix: Map labels and convert `y` to numeric type (int)
y = y.map({1: 1, 2: -1}).astype(int).to_numpy()  # Adjust mapping and convert to integer array

# Convert X to a numpy array
X = X.to_numpy()

# Check the final shapes of X and y
print(X.shape)
print(y.shape)


(570, 10)
(570,)


#### Shuffle Data

In [18]:
# Combine X and y into a single array
X_and_Y = np.hstack([X, y.reshape(-1, 1)])  # Combine features and labels

# Shuffle the combined array
np.random.seed(1)  # Set random seed for reproducibility
np.random.shuffle(X_and_Y)  # Shuffle rows

# Separate X and y after shuffling
X = X_and_Y[:, :-1]  # All columns except the last (features)
y = X_and_Y[:, -1]   # The last column (target labels)
y = y.astype(int)

# Check final shapes of X and y
print("X shape:", X.shape)  # Features shape
print("y shape:", y.shape)  # Labels shape


X shape: (570, 10)
y shape: (570,)


### Classifier #1: Decision Trees

In [19]:
# Hyperparameters - need to use cross validation and compute validation accuracy
param_grid = {
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 10, 20]
}

# Hyperparamter tuning in order to get the best hyperparameters - perform a 3-fold cross-validation
clf = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy', return_train_score=True)
grid_search.fit(X, y)

# Extract and display the validation accuracy for each combination
print("Validation Accuracies for Each Hyperparameter Combination:")
print("=" * 50)
cv_results = grid_search.cv_results_
for mean_val_acc, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Parameters: {params}, Validation Accuracy: {mean_val_acc:.4f}")

# Best hyperparameters based on validation accuracy
best_params = grid_search.best_params_
best_validation_accuracy = grid_search.best_score_
print("\nBest Hyperparameters:")
print(f"  {best_params}")
print(f"Validation Accuracy with Best Hyperparameters: {best_validation_accuracy:.4f}")

Validation Accuracies for Each Hyperparameter Combination:
Parameters: {'max_depth': 5, 'min_samples_split': 2}, Validation Accuracy: 0.6526
Parameters: {'max_depth': 5, 'min_samples_split': 10}, Validation Accuracy: 0.6509
Parameters: {'max_depth': 5, 'min_samples_split': 20}, Validation Accuracy: 0.6491
Parameters: {'max_depth': 10, 'min_samples_split': 2}, Validation Accuracy: 0.6596
Parameters: {'max_depth': 10, 'min_samples_split': 10}, Validation Accuracy: 0.6456
Parameters: {'max_depth': 10, 'min_samples_split': 20}, Validation Accuracy: 0.6474
Parameters: {'max_depth': None, 'min_samples_split': 2}, Validation Accuracy: 0.6509
Parameters: {'max_depth': None, 'min_samples_split': 10}, Validation Accuracy: 0.6351
Parameters: {'max_depth': None, 'min_samples_split': 20}, Validation Accuracy: 0.6246

Best Hyperparameters:
  {'max_depth': 10, 'min_samples_split': 2}
Validation Accuracy with Best Hyperparameters: 0.6596


In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Partition splits with 80/20, 50/50, 20/80
splits = [(.8, .2), (.5, .5), (.2, .8)]
results = {}
split_results = []  # Store split-wise metrics

# Outer loop for each partition
for split in splits:
    train_size, test_size = split
    trial_accuracies = {'train': [], 'test': []}
    trial_precisions = []
    trial_recalls = []
    trial_f1_scores = []

    # Inner loop to run 3 trials per split
    for trial in range(3): 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None)

        # Retrain the model with the best hyperparameters
        best_clf = DecisionTreeClassifier(
            max_depth=best_params['max_depth'], 
            min_samples_split=best_params['min_samples_split'], 
            random_state=42
        )
        best_clf.fit(X_train, y_train)

        # Compute train accuracy
        y_train_pred = best_clf.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        trial_accuracies['train'].append(train_accuracy)

        # Compute test accuracy
        y_test_pred = best_clf.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        trial_accuracies['test'].append(test_accuracy)

        # Calculate precision, recall, and F1-score
        trial_precisions.append(precision_score(y_test, y_test_pred, average="weighted", zero_division=0))
        trial_recalls.append(recall_score(y_test, y_test_pred, average="weighted"))
        trial_f1_scores.append(f1_score(y_test, y_test_pred, average="weighted"))

    # Average metrics over the 3 trials for the current split
    avg_train_accuracy = np.mean(trial_accuracies['train'])
    avg_test_accuracy = np.mean(trial_accuracies['test'])
    avg_precision = np.mean(trial_precisions)
    avg_recall = np.mean(trial_recalls)
    avg_f1_score = np.mean(trial_f1_scores)

    # Save split-level metrics
    split_results.append({
        'Split': f"{int(train_size * 100)}/{int(test_size * 100)}",
        'Test Accuracy': avg_test_accuracy,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1-Score': avg_f1_score
    })

    # Print split-specific results
    print(f"Results for {int(train_size * 100)}/{int(test_size * 100)} Split:")
    print(f"Train Accuracy: {avg_train_accuracy:.4f}")
    print(f"Test Accuracy: {avg_test_accuracy:.4f}")
    # print(f"Precision: {avg_precision:.4f}")
    # print(f"Recall: {avg_recall:.4f}")
    # print(f"F1-Score: {avg_f1_score:.4f}")
    print("-" * 50)

# # Save the metrics to use later for the summary table
# print("Overall Metrics for Decision Tree Classifier (Autism Dataset):")
# print(f"Overall Accuracy: {overall_accuracy_1:.4f}")
# print(f"Overall Precision: {overall_precision_1:.4f}")
# print(f"Overall Recall: {overall_recall_1:.4f}")
# print(f"Overall F1-Score: {overall_f1_score_1:.4f}")

Results for 80/20 Split:
Train Accuracy: 0.9123
Test Accuracy: 0.6257
--------------------------------------------------
Results for 50/50 Split:
Train Accuracy: 0.9439
Test Accuracy: 0.6199
--------------------------------------------------
Results for 20/80 Split:
Train Accuracy: 0.9825
Test Accuracy: 0.6608
--------------------------------------------------


#### Results for Decision Trees Classifier

In [21]:
# Calculate overall metrics for the Decision Tree classifier
overall_accuracy_1 = np.mean([r['Test Accuracy'] for r in split_results])
overall_precision_1 = np.mean([r['Precision'] for r in split_results])
overall_recall_1 = np.mean([r['Recall'] for r in split_results])
overall_f1_score_1 = np.mean([r['F1-Score'] for r in split_results])
# Create a formatted table for the split results
print("Decision Tree Classifier Results on Liver Dataset:")
print(f"{'Split':<10}{'ACC':<15}{'Precision':<15}{'Recall':<15}{'F1-Score':<15}")
print("-" * 65)

# Loop through the split results and display each row
for result in split_results:
    print(f"{result['Split']:<10}{result['Test Accuracy']:<15.4f}{result['Precision']:<15.4f}{result['Recall']:<15.4f}{result['F1-Score']:<15.4f}")

# Print the overall metrics as the last row
print("-" * 65)
print(f"{'Overall':<10}{overall_accuracy_1:<15.4f}{overall_precision_1:<15.4f}{overall_recall_1:<15.4f}{overall_f1_score_1:<15.4f}")

Decision Tree Classifier Results on Liver Dataset:
Split     ACC            Precision      Recall         F1-Score       
-----------------------------------------------------------------
80/20     0.6257         0.6534         0.6257         0.6372         
50/50     0.6199         0.6127         0.6199         0.6154         
20/80     0.6608         0.6658         0.6608         0.6621         
-----------------------------------------------------------------
Overall   0.6355         0.6440         0.6355         0.6382         


### Classifier #2: SVM (with RBF kernel)

In [22]:
# Define the parameter grid for SVM
param_grid = {
    'C': [1, 10, 100, 1000, 10000],
    'gamma': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'kernel': ['rbf']
}

# Set up the SVM model
svm = SVC()

# Perform GridSearchCV on the entire dataset (no need for X_train, Y_train if you're not splitting beforehand)
grid_search = GridSearchCV(svm, param_grid, cv=3, scoring='accuracy', return_train_score=True)  # 3-fold cross-validation

# Fit the model (this performs the search)
grid_search.fit(X, y)  # Use the entire dataset here, not just the training set

# Extract the results from the grid search
cv_results = grid_search.cv_results_

# Print the validation accuracies for each hyperparameter combination
print("Validation Accuracies for Each Hyperparameter Combination:")
print("=" * 50)
for mean_val_acc, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Parameters: {params}, Validation Accuracy: {mean_val_acc:.4f}")

# Best hyperparameters based on validation accuracy
best_params = grid_search.best_params_
best_validation_accuracy = grid_search.best_score_

# Print the best hyperparameters and the corresponding validation accuracy
print("\nBest Hyperparameters:")
print(f"  {best_params}")
print(f"Validation Accuracy with Best Hyperparameters: {best_validation_accuracy:.4f}")

Validation Accuracies for Each Hyperparameter Combination:
Parameters: {'C': 1, 'gamma': 1e-06, 'kernel': 'rbf'}, Validation Accuracy: 0.7123
Parameters: {'C': 1, 'gamma': 1e-05, 'kernel': 'rbf'}, Validation Accuracy: 0.7123
Parameters: {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}, Validation Accuracy: 0.7123
Parameters: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}, Validation Accuracy: 0.7140
Parameters: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}, Validation Accuracy: 0.7105
Parameters: {'C': 10, 'gamma': 1e-06, 'kernel': 'rbf'}, Validation Accuracy: 0.7123
Parameters: {'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}, Validation Accuracy: 0.7053
Parameters: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}, Validation Accuracy: 0.6947
Parameters: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}, Validation Accuracy: 0.6754
Parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}, Validation Accuracy: 0.6930
Parameters: {'C': 100, 'gamma': 1e-06, 'kernel': 'rbf'}, Validation Accuracy: 0.7123
Parameters: {'C': 100

In [31]:
# Best hyperparameters obtained from GridSearchCV
best_C = 1
best_gamma = 0.001

# Split ratios
split_ratios = [(0.8, 0.2), (0.5, 0.5), (0.2, 0.8)]
split_results_svm_2 = []

# Iterate over splits
for train_size, test_size in split_ratios:
    trial_train_accuracies = []
    trial_validation_accuracies = []
    trial_test_accuracies = []
    trial_precisions = []
    trial_recalls = []
    trial_f1_scores = []
    
    for _ in range(3):  # Perform 3 trials for each split
        # Split the data into train and temp (validation + test)
        X_train, X_temp, Y_train, Y_temp = train_test_split(X, y, train_size=train_size, stratify=y, random_state=None)
        
        # Now split the temp data into validation and test
        X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=test_size, stratify=Y_temp, random_state=None)

        # Train the SVM model using the best hyperparameters
        clf = SVC(C=best_C, gamma=best_gamma, kernel='rbf')
        clf.fit(X_train, Y_train)

        # Calculate train accuracy
        train_accuracy = clf.score(X_train, Y_train)
        trial_train_accuracies.append(train_accuracy)

        # Calculate validation accuracy
        validation_accuracy = clf.score(X_val, Y_val)
        trial_validation_accuracies.append(validation_accuracy)

        # Calculate test accuracy
        test_accuracy = clf.score(X_test, Y_test)
        trial_test_accuracies.append(test_accuracy)

        # Calculate precision, recall, and F1-score
        Y_test_pred = clf.predict(X_test)
        trial_precisions.append(precision_score(Y_test, Y_test_pred, average="weighted", zero_division=0))
        trial_recalls.append(recall_score(Y_test, Y_test_pred, average="weighted"))
        trial_f1_scores.append(f1_score(Y_test, Y_test_pred, average="weighted"))

    # Calculate and store split-wise average metrics
    avg_train_accuracy = np.mean(trial_train_accuracies)
    avg_validation_accuracy = np.mean(trial_validation_accuracies)
    avg_test_accuracy = np.mean(trial_test_accuracies)
    avg_precision = np.mean(trial_precisions)
    avg_recall = np.mean(trial_recalls)
    avg_f1_score = np.mean(trial_f1_scores)

    split_results_svm_2.append({
        'Split': f"{int(train_size * 100)}/{int(test_size * 100)}",
        'ACC': avg_test_accuracy,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1-Score': avg_f1_score
    })

    # Print split-specific results
    print(f"\nResults for {int(train_size * 100)}/{int(test_size * 100)} Split:")
    print(f"Train Accuracy: {avg_train_accuracy:.4f}")
    print(f"Test Accuracy: {avg_test_accuracy:.4f}")
    # print(f"Precision: {avg_precision:.4f}")
    # print(f"Recall: {avg_recall:.4f}")
    # print(f"F1-Score: {avg_f1_score:.4f}")
    print("-" * 50)


Results for 80/20 Split:
Train Accuracy: 0.7763
Test Accuracy: 0.6812
--------------------------------------------------

Results for 50/50 Split:
Train Accuracy: 0.8012
Test Accuracy: 0.7110
--------------------------------------------------

Results for 20/80 Split:
Train Accuracy: 0.8158
Test Accuracy: 0.6968
--------------------------------------------------


#### Results for SVM Classifier

In [32]:
# Calculate overall metrics for the SVM classifier
overall_accuracy_2 = np.mean([r['ACC'] for r in split_results_svm_2])
overall_precision_2 = np.mean([r['Precision'] for r in split_results_svm_2])
overall_recall_2 = np.mean([r['Recall'] for r in split_results_svm_2])
overall_f1_score_2 = np.mean([r['F1-Score'] for r in split_results_svm_2])

# Create a formatted table for the split results
print("SVM Classifier Results on Liver Dataset:")
print(f"{'Split':<10}{'ACC':<15}{'Precision':<15}{'Recall':<15}{'F1-Score':<15}")
print("-" * 75)

# Loop through the split results and display each row
for result in split_results_svm_2:
    print(f"{result['Split']:<10}{result['ACC']:<15.4f}{result['Precision']:<15.4f}{result['Recall']:<15.4f}{result['F1-Score']:<15.4f}")

# Print the overall metrics as the last row
print("-" * 75)
print(f"{'Overall':<10}{overall_accuracy_2:<15.4f}{overall_precision_2:<15.4f}{overall_recall_2:<15.4f}{overall_f1_score_2:<15.4f}")

SVM Classifier Results on Liver Dataset:
Split     ACC            Precision      Recall         F1-Score       
---------------------------------------------------------------------------
80/20     0.6812         0.4807         0.6812         0.5636         
50/50     0.7110         0.6728         0.7110         0.6527         
20/80     0.6968         0.6095         0.6968         0.6129         
---------------------------------------------------------------------------
Overall   0.6963         0.5877         0.6963         0.6097         


### Classifier #3: Logistic Regression

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Assuming `X_train`, `y_train` are already defined.

# Hyperparameter tuning for Logistic Regression
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l2'], 'solver': ['lbfgs']}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train, y_train)

# Extract the results from the grid search
cv_results = grid_search.cv_results_

# Print the validation accuracies for each hyperparameter combination
print("Validation Accuracies for Each Hyperparameter Combination:")
print("=" * 50)
for mean_val_acc, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Parameters: {params}, Validation Accuracy: {mean_val_acc:.4f}")

# Best hyperparameters based on validation accuracy
best_params = grid_search.best_params_
best_validation_accuracy = grid_search.best_score_

# Print the best hyperparameters and the corresponding validation accuracy
print("\nBest Hyperparameters:")
print(f"  {best_params}")
print(f"Validation Accuracy with Best Hyperparameters: {best_validation_accuracy:.4f}")

Validation Accuracies for Each Hyperparameter Combination:
Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}, Validation Accuracy: 0.7107
Parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}, Validation Accuracy: 0.7020
Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}, Validation Accuracy: 0.6933
Parameters: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}, Validation Accuracy: 0.6759
Parameters: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}, Validation Accuracy: 0.6846

Best Hyperparameters:
  {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
Validation Accuracy with Best Hyperparameters: 0.7107


In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

best_C = 1
best_penalty = 'l2'
best_solver = 'lbfgs'

# Assuming `X` and `y` are your feature and target arrays
# Split ratios
split_ratios = [(0.8, 0.2), (0.5, 0.5), (0.2, 0.8)]

# Lists for storing accuracies and other metrics
split_results_logreg = []

# Iterate over splits
for train_size, test_size in split_ratios:
    trial_train_accuracies = []
    trial_validation_accuracies = []
    trial_test_accuracies = []
    trial_precisions = []
    trial_recalls = []
    trial_f1_scores = []
    
    for _ in range(3):  # Running 3 trials
        # Split the data into train and temp (validation + test)
        X_train, X_temp, Y_train, Y_temp = train_test_split(X, y, train_size=train_size, random_state=None)
        
        # Now split the temp data into validation and test
        X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=test_size, random_state=None)

        # Train the Logistic Regression model using the best hyperparameters
        clf = LogisticRegression(C=best_C, penalty=best_penalty, solver=best_solver, max_iter=10000)
        clf.fit(X_train, Y_train)

        # Calculate the accuracy on the training set
        train_accuracy = clf.score(X_train, Y_train)
        trial_train_accuracies.append(train_accuracy)

        # Calculate the accuracy on the validation set
        validation_accuracy = clf.score(X_val, Y_val)
        trial_validation_accuracies.append(validation_accuracy)

        # Calculate the accuracy on the test set
        test_accuracy = clf.score(X_test, Y_test)
        trial_test_accuracies.append(test_accuracy)

        # Calculate precision, recall, and F1-score on the test set
        Y_test_pred = clf.predict(X_test)
        precision = precision_score(Y_test, Y_test_pred, average="weighted", zero_division=0)
        recall = recall_score(Y_test, Y_test_pred, average="weighted")
        f1 = f1_score(Y_test, Y_test_pred, average="weighted")
        trial_precisions.append(precision)
        trial_recalls.append(recall)
        trial_f1_scores.append(f1)

    # Calculate and store average metrics for the current split
    avg_train_accuracy = np.mean(trial_train_accuracies)
    avg_validation_accuracy = np.mean(trial_validation_accuracies)
    avg_test_accuracy = np.mean(trial_test_accuracies)
    avg_precision = np.mean(trial_precisions)
    avg_recall = np.mean(trial_recalls)
    avg_f1_score = np.mean(trial_f1_scores)

    split_results_logreg.append({
        'Split': f"{int(train_size * 100)}/{int(test_size * 100)}",
        'Test Accuracy': avg_test_accuracy,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1-Score': avg_f1_score
    })

    # Print split-specific results
    print(f"\nResults for {int(train_size * 100)}/{int(test_size * 100)} Split:")
    print(f"Avg Train Accuracy = {avg_train_accuracy:.4f}")
    print(f"Avg Test Accuracy = {avg_test_accuracy:.4f}")
    print("-" * 50)


Results for 80/20 Split:
Avg Train Accuracy = 0.7376
Avg Test Accuracy = 0.6377
--------------------------------------------------

Results for 50/50 Split:
Avg Train Accuracy = 0.7392
Avg Test Accuracy = 0.6993
--------------------------------------------------

Results for 20/80 Split:
Avg Train Accuracy = 0.7661
Avg Test Accuracy = 0.7123
--------------------------------------------------


#### Results for Logistic Regression Classifier

In [34]:
# Calculate overall metrics
overall_accuracy_3 = np.mean([r['Test Accuracy'] for r in split_results_logreg])
overall_precision_3 = np.mean([r['Precision'] for r in split_results_logreg])
overall_recall_3 = np.mean([r['Recall'] for r in split_results_logreg])
overall_f1_score_3 = np.mean([r['F1-Score'] for r in split_results_logreg])

# Create a formatted table for the split results
print("\nLogistic Regression Classifier Results on Liver Dataset:")
print(f"{'Split':<10}{'ACC':<15}{'Precision':<15}{'Recall':<15}{'F1-Score':<15}")
print("-" * 70)

# Loop through the split results and display each row
for result in split_results_logreg:
    print(f"{result['Split']:<10}{result['Test Accuracy']:<15.4f}{result['Precision']:<15.4f}{result['Recall']:<15.4f}{result['F1-Score']:<15.4f}")

# Print the overall metrics as the last row
print("-" * 70)
print(f"{'Overall':<10}{overall_accuracy_3:<15.4f}{overall_precision_3:<15.4f}{overall_recall_3:<15.4f}{overall_f1_score_3:<15.4f}")



Logistic Regression Classifier Results on Liver Dataset:
Split     ACC            Precision      Recall         F1-Score       
----------------------------------------------------------------------
80/20     0.6377         0.6525         0.6377         0.5880         
50/50     0.6993         0.6685         0.6993         0.6721         
20/80     0.7123         0.6878         0.7123         0.6923         
----------------------------------------------------------------------
Overall   0.6831         0.6696         0.6831         0.6508         


## Final Summary Table

In [36]:
summary_data = {
    'Classifier': ['Decision Tree', 'Support Vector Machines', 'Logistic Regression'],  # Add names for all classifiers
    'ACC': [overall_accuracy_1, overall_accuracy_2, overall_accuracy_3],  # Add more as needed
    'Precision': [overall_precision_1, overall_precision_2, overall_precision_3],
    'Recall': [overall_recall_1, overall_recall_2, overall_recall_3],
    'F1-Score': [overall_f1_score_1, overall_f1_score_2, overall_f1_score_3]
}

# Convert to DataFrame for cleaner display
import pandas as pd
summary_df = pd.DataFrame(summary_data)

# Print the summary table
print("Final Summary Table of Overall Metrics:")
print(summary_df)

Final Summary Table of Overall Metrics:
                Classifier       ACC  Precision    Recall  F1-Score
0            Decision Tree  0.635478   0.643981  0.635478  0.638238
1  Support Vector Machines  0.696306   0.587669  0.696306  0.609749
2      Logistic Regression  0.683104   0.669586  0.683104  0.650806
